# Analyzing Aggravated Burglaries in Davidson County

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from IPython.display import Image
import json
import geopandas as gpd
from io import StringIO

Create a choropleth showing the number of burglaries per 1000 residents for each census tract.

Finally, we'll build some statistical models to see how well we can explain the number of aggravated burglaries using the median income of each census tract. For this, we'll be using the Generalized Linear Models module of the statsmodels library.

a. Build a "base model" - a Poisson regression model with just an intercept term with target variable the rate of burglaries per census tract. (Offset using the [log of the] population so that we are looking at the rate of burglaries per population instead of the number of burglaries.)

b. Now, build a Poisson regression model with target variable the rate of burglaries and predictor variable the median income. (Don't forget to offset by the population).

c. Finally, try out a negative binomial model. To get started with a negative binomial model, you can check out this tutorial.

d. How do your models compare? Hint: the fit models have an AIC attribute.

Prepare a short presentation (<10 minutes) of your findings.

Additional Resources for Generalized Linear Models:

DataCamp - Generalized Linear Models in Python
Beyond Multiple Linear Regression, Chapter 4 Warning - the code in this book is all R, but the conceptual explanations are very clear.
This set of notes, which talks about the problem of overdispersion.

1. A dataset containing details about Metro Nashville Police Department reported incidents is available at https://data.nashville.gov/Police/Metro-Nashville-Police-Department-Incidents/2u6v-ujjs. Make use of the API to find all aggravated burglary incidents that were reported during the six month period from January 1, 2021 through June 30, 2021.

In [38]:
endpoint = 'https://data.nashville.gov/resource/2u6v-ujjs.geojson'
params = {'offense_description':'BURGLARY- AGGRAVATED',
              '$where':'incident_occurred between \'2021-01-01\' and \'2021-06-30\'',
              '$limit':2000
              }
response = requests.get(endpoint, params = params)

In [41]:
agg_burg = gpd.read_file(StringIO(response.text))

In [48]:
# drop duplicate rows with the same incident_number
agg_burg=agg_burg.drop_duplicates(subset = ['incident_number'])

2. Download the 2019 census tract shapefiles for Tennessee from https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.2019.html. (The FIPS code for Tennessee is 47). Perform a spatial join to determine the census tract in which each burglary incident occurred. Which census tract had the highest number of burglaries? Warning - each incident can appear multiple times if there are multiple victims, so be sure that you aren't double-counting any incidents.

In [49]:
census_tract=gpd.read_file('../data/tl_2019_47_tract/')

In [52]:
census_tract

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,47,165,021002,47165021002,210.02,Census Tract 210.02,G5020,S,30654374,207994,+36.3521602,-086.6464309,"POLYGON ((-86.69927 36.32775, -86.69923 36.327..."
1,47,165,020901,47165020901,209.01,Census Tract 209.01,G5020,S,23811766,3412697,+36.3483143,-086.4070907,"POLYGON ((-86.43949 36.35962, -86.43915 36.359..."
2,47,165,020300,47165020300,203,Census Tract 203,G5020,S,14629292,0,+36.5796910,-086.5114955,"POLYGON ((-86.53716 36.56852, -86.53713 36.568..."
3,47,165,020902,47165020902,209.02,Census Tract 209.02,G5020,S,9756715,0,+36.3724350,-086.4411274,"POLYGON ((-86.47106 36.38238, -86.47081 36.382..."
4,47,165,020800,47165020800,208,Census Tract 208,G5020,S,10235406,37743,+36.4005117,-086.4528100,"POLYGON ((-86.48786 36.37320, -86.48783 36.373..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1492,47,157,022112,47157022112,221.12,Census Tract 221.12,G5020,S,3293232,0,+35.0282414,-090.0009887,"POLYGON ((-90.01257 35.02328, -90.01256 35.023..."
1493,47,073,050501,47073050501,505.01,Census Tract 505.01,G5020,S,83253173,0,+36.5659218,-082.7714427,"POLYGON ((-82.85011 36.54107, -82.84955 36.541..."
1494,47,073,050601,47073050601,506.01,Census Tract 506.01,G5020,S,23852225,177932,+36.5836443,-082.6318663,"POLYGON ((-82.68939 36.58793, -82.68936 36.588..."
1495,47,073,050602,47073050602,506.02,Census Tract 506.02,G5020,S,23381992,668716,+36.5467218,-082.6458382,"POLYGON ((-82.69446 36.54856, -82.69445 36.548..."


In [53]:
burg_census = gpd.sjoin(agg_burg, census_tract, op = 'within')

/var/folders/h3/4dwpklsd0vq23n7lrrx_lryr0000gn/T/ipykernel_7204/1384890642.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPSG:4269

  burg_census = gpd.sjoin(agg_burg, census_tract, op = 'within')


In [63]:
burg_census['NAMELSAD'].value_counts().index[0]

'Census Tract 163'

3. For this part, you'll need to request a census API key. Using the 2019 American Community Survey API, obtain, for each census tract, the population (B01001_001E in the detailed tables) and the median income (S1901_C01_012E in the subject tables). Hint: Tennessee's FIPS code is 47 and Davidson County's FIPS code is 37. Merge this new data with the burglaries data above.

In [103]:
with open('../data/census.json') as fi:
    credentials = json.load(fi)
api_key = credentials['api_key']